In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

# 策略

In [21]:
class Strategy_comp1(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('p_stake', 100),
        ('wave_period', 7),
        ('wave_slen', 3),
        ('using_atr', False),
        ('atr_initial', 5000),
        ('atr_period', 14),
        ('atr_percent', 1),
        ('atr_risk', 1),
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('buy_signal', 'b'),
        ('sell_signal', 'b'),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.buyprice = None
        self.sellprice = None
        self.order = None

        # Add a Wave indicator
        ## Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
                    self.datas[0], period=self.params.wave_period)
        
        ## Add a Wave singal stack
        self.stack = [0] * self.params.wave_slen
        
        # Add MACD indicator
        self.macdhisto = bt.indicators.MACDHisto(self.datas[0])

        # Adding an ATR indicator
        self.tr = bt.indicators.Max((self.datahigh - self.datalow), 
                                                         abs(self.dataclose(-1) - self.datahigh),
                                                         abs(self.dataclose(-1) - self.datalow))
        self.atr = bt.indicators.SimpleMovingAverage(self.tr, period=self.params.atr_period)
        
        # Add Bollinger Bands indicator
        self.lines.top = bt.indicators.BollingerBands(self.datas[0], 
                                                      period = self.params.bollinger_per, 
                                                      devfactor = self.params.bollinger_dev).top
        self.lines.bot = bt.indicators.BollingerBands(self.datas[0], 
                                                      period = self.params.bollinger_per, 
                                                      devfactor = self.params.bollinger_dev).bot

        # Add training analysis
        self.profit_array = np.zeros(3)
        self.position_array = np.zeros(3)
        self.execute_amount = None


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.signal_array[2] == 1:
                    self.buyprice = order.executed.price
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:  # Sell
                self.sellprice = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        #self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    

    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Signal Detection
        self.signal_array = np.zeros(3)
        ## Bollinger Signal - array index 0
        ### Bollinger Buy Signal
        if (self.dataclose[0] > self.lines.top[0] and self.dataclose[-1] < self.lines.top[-1]) or (self.dataclose[0] > self.lines.bot[0] and self.dataclose[-1] < self.lines.bot[-1]):
            self.signal_array[0] = 1
        ### Bollinger Sell Singal
        elif  (self.dataclose[0] < self.lines.top[0] and self.dataclose[-1] > self.lines.top[-1]) or (self.dataclose[0] < self.lines.bot[0] and self.dataclose[-1] > self.lines.bot[-1]):
            if self.getposition(self.data).size > 0:
                self.signal_array[0] = -1


        ## MACD Signal - array index 1
        ### MACD Buy Signal
        if self.macdhisto.histo[0] > 0 and self.macdhisto.histo[-1] < 0:
            self.signal_array[1] = 1
                   
        ### MACD Sell Signal
        elif self.macdhisto.histo[0] < 0 and self.macdhisto.histo[-1] > 0:
                if self.getposition(self.data).size > 0:
                    self.signal_array[1] = -1


        ## Wave Signal - array index 2
        ### Wave Buy Signal        
        for i in range(1, self.params.wave_slen+1):
            self.stack[-i] = 1 if self.sma[-i+1] - self.sma[-i] > 0 else -1

        if self.stack[-1] == 1 and sum(self.stack) in [-1 * (self.params.wave_slen - 2), -1 * (self.params.wave_slen - 3)]:
            #### Avoid long position in Downward wave
            if self.buyprice is None:
                self.signal_array[2] = 1
            elif self.dataclose > self.buyprice:
                self.signal_array[2] = 1

        ### Wave Sell Signal
        elif self.stack[-1] == -1 and sum(self.stack) in [1 * (self.params.wave_slen - 2), 1 * (self.params.wave_slen - 3)]:
                if self.getposition(self.data).size > 0:
                    self.signal_array[2] = -1
        
        
        # Buy/Sell
        if (self.params.buy_signal == 'b' and self.signal_array[0] == 1) or (self.params.buy_signal == 'm' and self.signal_array[1] == 1) or (self.params.buy_signal == 'w' and self.signal_array[2] == 1):
            if self.params.using_atr:
                opt_pos = np.round(self.params.atr_initial * self.params.atr_percent / self.params.p_stake / self.atr[0])
                if self.getposition(self.data).size == 0:
                    self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                        opt_pos*self.params.p_stake,
                        self.getposition(self.data).size))
                    self.buy(size=opt_pos*self.params.p_stake)
                elif self.getposition(self.data).size < opt_pos*self.params.p_stake:    
                    self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                            opt_pos*self.params.p_stake-self.getposition(self.data).size, 
                            self.getposition(self.data).size))
                    self.buy(size=opt_pos*self.params.p_stake-self.getposition(self.data).size)
                elif self.getposition(self.data).size > opt_pos*self.params.p_stake:
                    self.log('SELL CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                            self.getposition(self.data).size-opt_pos*self.params.p_stake, 
                            self.getposition(self.data).size))
                    self.sell(size=self.getposition(self.data).size-opt_pos*self.params.p_stake)
            else:
                self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                        self.params.p_stake,
                        self.getposition(self.data).size))
                self.buy(size=self.params.p_stake)
        
        elif (self.params.sell_signal == 'b' and self.signal_array[0] == -1) or(self.params.sell_signal == 'm' and self.signal_array[1] == -1) or (self.params.sell_signal == 'w' and self.signal_array[2] == -1):
            self.log('Sell CREATE(Close), Price: %.2f, Current Position: %i' % (self.dataclose[0], 
                     self.getposition(self.data).size))
            self.close()


    def stop(self):
        self.log('Ending Position:%i. Ending Value:%.2f.' %
                (self.getposition(self.data).size,
                self.cerebro.broker.getvalue()), doprint=False)

# 循环回测

In [29]:
if __name__ ==  '__main__':
    buy_s = ['b','m','w']
    sell_s = ['b','m','w']
    profit_dict = {}
    for s in product(*[buy_s, sell_s]):
            
        # Create a cerebro entity
        cerebro = bt.Cerebro()

        # Add a strategy
        strats = cerebro.addstrategy(Strategy_comp1, printlog=False, using_atr=True, atr_percent=0.01, buy_signal = s[0], sell_signal = s[1]) 

        s_date = datetime.date(2021,12,31) - datetime.timedelta(days=365*3)
        e_date = datetime.date(2021,12,31)

        # Create stock Data Feed
        stock_index = '002057.SZ'
        df = utilsJ.stock_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                            stock_index, s_date, e_date)
        data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

        # Add the index Data Feed to Cerebo
        cerebro.adddata(data)
        
        # Set cash inside the strategy
        cerebro.broker = bt.brokers.BackBroker(coc=True)   
        cerebro.broker.setcash(5000)

        # Set commission
        #cerebro.broker.setcommission()

        # Print out the starting conditions
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Run over everything
        cerebro.run()

        # Print out the final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
        profit_dict[s] = (final_value - start_value) / start_value * 100
    
    print(sorted(profit_dict.items(), key=lambda x: x[1], reverse=True))

[(('b', 'm'), 22.14), (('m', 'm'), 20.580000000000002), (('m', 'b'), 19.42), (('b', 'b'), 16.72), (('b', 'w'), 14.16), (('m', 'w'), 11.06), (('w', 'b'), 8.799999999999999), (('w', 'm'), 0.5599999999999999), (('w', 'w'), -0.32)]


# 报告

In [31]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(Strategy_comp1, printlog=True, using_atr=True, atr_percent=0.01, buy_signal = 'b', sell_signal = 'm') 

    s_date = datetime.date(2021,12,31) - datetime.timedelta(days=365*3)
    e_date = datetime.date(2021,12,31)

    # Create stock Data Feed
    stock_index = '002057.SZ'
    df = utilsJ.stock_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                        stock_index, s_date, e_date)
    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)
    
    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(5000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

Starting Portfolio Value: 5000.00
2019-03-14: BUY CREATE, Price: 7.72, Lots: 200, Current Position: 0
2019-03-15: BUY EXECUTED, Price:7.72, Lot:200, Cash:3456.
2019-03-26: Sell CREATE(Close), Price: 7.61, Current Position: 200
2019-03-27: SELL EXECUTED, Price:7.61, Lot:200, Cash:4978.
2019-03-27: OPERATION PROFIT, GROSS -22.00, NET -22.00
2019-04-16: BUY CREATE, Price: 8.76, Lots: 200, Current Position: 0
2019-04-17: BUY EXECUTED, Price:8.76, Lot:200, Cash:3226.
2019-05-09: Sell CREATE(Close), Price: 10.50, Current Position: 200
2019-05-10: SELL EXECUTED, Price:10.50, Lot:200, Cash:5326.
2019-05-10: OPERATION PROFIT, GROSS 348.00, NET 348.00
2019-05-15: BUY CREATE, Price: 13.86, Lots: 0, Current Position: 0
2019-06-13: BUY CREATE, Price: 18.66, Lots: 0, Current Position: 0
2019-06-19: BUY CREATE, Price: 12.27, Lots: 0, Current Position: 0
2019-09-10: BUY CREATE, Price: 11.00, Lots: 100, Current Position: 0
2019-09-11: BUY EXECUTED, Price:11.00, Lot:100, Cash:4226.
2019-09-24: Sell CREA

In [32]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]